In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoLarsIC
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from math import log

In [4]:
df = pd.read_csv('/Users/joaopedro/Documents/MSBA/Classes/Winter Quarter/BAX 452 - Machine Learning/Assignments/04. Python - Lasso Regression/GitHub/cars.csv')

df.head()

,Unnamed: 0,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,0,18.0,8,307.0,130.0,3504,12.0,70,1
1,1,15.0,8,350.0,165.0,3693,11.5,70,1
2,2,18.0,8,318.0,150.0,3436,11.0,70,1
3,3,16.0,8,304.0,150.0,3433,12.0,70,1
4,4,17.0,8,302.0,140.0,3449,10.5,70,1


## LASSO

**Create Lasso regression models to predict the ‘mpg’ for a car using 20% and 80% of the training data for the 2 models.**

In [3]:
df.nunique()

Unnamed: 0      398
mpg             129
cylinders         5
displacement     82
horsepower       94
weight          351
acceleration     95
model year       13
origin            3
dtype: int64

In [4]:
## Cols to Use
cols_to_use = ["mpg","cylinders","displacement","horsepower","weight","acceleration","model year","origin"]

df = df[cols_to_use]

df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


In [5]:
## Check for NA
df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    float64
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 25.0 KB


In [7]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin'],
      dtype='object')

In [8]:
## Standardizing the data
scaler = StandardScaler()
df[['cylinders', 'displacement', 'horsepower',
     'weight', 'acceleration', 'model year', 'origin']] = scaler.fit_transform(df[['cylinders',
                                                                              'displacement',
                                                                              'horsepower',
                                                                              'weight',
                                                                              'acceleration',
                                                                              'model year', 'origin']])

df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,1.498191,1.090604,0.673118,0.630870,-1.295498,-1.627426,-0.715145
1,15.0,1.498191,1.503514,1.589958,0.854333,-1.477038,-1.627426,-0.715145
2,18.0,1.498191,1.196232,1.197027,0.550470,-1.658577,-1.627426,-0.715145
3,16.0,1.498191,1.061796,1.197027,0.546923,-1.295498,-1.627426,-0.715145
4,17.0,1.498191,1.042591,0.935072,0.565841,-1.840117,-1.627426,-0.715145


In [9]:
## Getting X and Y
x = df.drop('mpg', axis = 1)
y = df['mpg']

In [10]:
## Train Test Split - Test 20%
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [11]:
## Create Lasso Regression
reg = Lasso(alpha=1).fit(X_train, y_train)

print('Lasso Regression: R^2 score on training set', reg.score(X_train, y_train)*100)
print('Lasso Regression: R^2 score on test set', reg.score(X_test, y_test)*100)

Lasso Regression: R^2 score on training set 78.2029699424824
Lasso Regression: R^2 score on test set 76.84830004927574


In [12]:
## Train Test Split - Test 80%
X_train2, X_test2, y_train2, y_test2 = train_test_split(x, y, test_size = 0.8)

In [13]:
## Create Lasso Regression
reg2 = Lasso(alpha=1).fit(X_train2, y_train2)

print('Lasso Regression: R^2 score on training set', reg2.score(X_train2, y_train2)*100)
print('Lasso Regression: R^2 score on test set', reg2.score(X_test2, y_test2)*100)

Lasso Regression: R^2 score on training set 75.96682848638284
Lasso Regression: R^2 score on test set 75.39293904958164


### Alpha Tunning

**For the above model, tune ‘alpha’ using the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) to find the optimum value for it. Explain how that value for alpha was chosen. Build a model using that value of alpha. [Hint: Have a look at sklearn’s Lasso model with Lars]**

In [14]:
## Lasso Lars AIC - Test 20%
lasso_lars_aic = LassoLarsIC(criterion = "aic", normalize = False).fit(X_train, y_train)
lasso_lars_bic = LassoLarsIC(criterion = "bic", normalize = False).fit(X_train, y_train)

In [15]:
## Store AIC metric for each alpha
results = pd.DataFrame(
    {
        "alphas": lasso_lars_aic.alphas_,
        "AIC criterion": lasso_lars_aic.criterion_,
        "BIC criterion": lasso_lars_bic.criterion_
    }
).set_index("alphas")
alpha_aic = lasso_lars_aic.alpha_
alpha_bic = lasso_lars_bic.alpha_

In [16]:
## Check which value leads to minimum AIC
def highlight_min(x):
    x_min = x.min()
    return ["font-weight: bold" if v == x_min else "" for v in x]

results.style.apply(highlight_min)

,AIC criterion,BIC criterion
alphas,,
6.380752659898435,318.000000,318.000000
3.7729992782557433,175.341754,179.103805
3.5016593808575167,166.002119,173.526222
1.3159894743293548,82.247788,93.533942
0.29550136158790596,68.669505,83.717710
0.06231785726160607,69.677409,88.487666
0.051577445170884514,71.300807,93.873115
0.0,71.909612,98.243972


In [17]:
## Create Lasso Regression with Optimum Alpha
reg_opt = Lasso(alpha=0.05669997460159893)
reg_opt.fit(X_train, y_train)

print('Lasso Regression: R^2 score on training set', reg_opt.score(X_train, y_train)*100)
print('Lasso Regression: R^2 score on test set', reg_opt.score(X_test, y_test)*100)

Lasso Regression: R^2 score on training set 81.29860980476292
Lasso Regression: R^2 score on test set 82.69425008456777


We used a 3 step approach to select the alpha:

1. We used LassoLarsIC to check the AIC and BIC for different alphas. 
2. We ploted the AIC and BIC values calculated for models with different alphas.
3. We selected the alpha value related to the lowest AIC and BIC

## Linear Regression with CV

**Build a simple regression model using 10-fold cross validation for the same data. Write your observations about the R-squared values for the models and their predictions.**

In [18]:
## Define CV method to use
cv = KFold(n_splits = 10, random_state = 1, shuffle = True)

## Build model
lm = LinearRegression().fit(x, y)

## Use kfold to evaluate model
r2_train = cross_val_score(lm, x, y, scoring='r2', cv = cv)

## Model score on test dataset
print('Linear Regression: R^2 score of all CV models', r2_train*100)
print('Linear Regression: mean R^2 score of CV models', r2_train.mean()*100)

Linear Regression: R^2 score of all CV models [87.45742632 84.94546423 81.75258875 83.40023347 74.15918327 80.59028059
 82.38334506 78.63852255 77.5818874  79.60073107]
Linear Regression: mean R^2 score of CV models 81.05096627069838


Using 10-fold Cross Validation, we found R2 ranging from 79.60 to 87.46 using the complete data set. The average value was 81.05

## Information Criteria Evaluation

**Calculate AIC, AICc (i.e. Corrected AIC) and BIC for the models you built in question 1 and question 2. According to each of the measures, which is the better model? Is BIC always greater than AIC? Please explain. Compare the AICs with the corresponding corrected AICs.**

In [19]:
## AIC
def calculate_aic(n, mse, num_params):
    aic = n * log(mse) + 2 * num_params
    return aic

def calculate_bic(n, mse, num_params):
    bic = n * log(mse) + num_params * log(n)
    return bic

def calculate_aicc(n, mse, num_params):
    aicc = n * log(mse) + 2 * num_params + (2 * num_params * num_params + 2 * num_params)/(n-num_params - 1)
    return aicc

## Num Params
num_params1a = len(reg.coef_)
num_params1b = len(reg2.coef_)
num_params2 = len(reg_opt.coef_)

## Predict training set
yhat1a = reg.predict(x)
yhat1b = reg2.predict(x)
yhat2 = reg_opt.predict(x)

## Calculate the error
mse1a = mean_squared_error(y, yhat1a)
mse1b = mean_squared_error(y, yhat1b)
mse2 = mean_squared_error(y, yhat2)

## Calculate the AIC, AICc, BIC
aic1a = calculate_aic(len(y), mse1a, num_params1a)
aic1b = calculate_aic(len(y), mse1b, num_params1b)
aic2 = calculate_aic(len(y), mse2, num_params2)

aicc1a = calculate_aicc(len(y), mse1a, num_params1a)
aicc1b = calculate_aicc(len(y), mse1b, num_params1b)
aicc2 = calculate_aicc(len(y), mse2, num_params2)

bic1a = calculate_bic(len(y), mse1a, num_params1a)
bic1b = calculate_bic(len(y), mse1b, num_params1b)
bic2 = calculate_bic(len(y), mse2, num_params2)

print('Model 1a: AIC: %.3f' % aic1a, 'AICc: %.3f' %aicc1a, 'BIC: %.3f' %bic1a)
print('Model 1b: AIC: %.3f' % aic1b, 'AICc: %.3f' %aicc1b, 'BIC: %.3f' %bic1b)
print('Model 2: AIC: %.3f' % aic2, 'AICc: %.3f' %aicc2, 'BIC: %.3f' %bic2)

Model 1a: AIC: 1047.510 AICc: 1047.797 BIC: 1075.415
Model 1b: AIC: 1090.150 AICc: 1090.437 BIC: 1118.055
Model 2: AIC: 974.445 AICc: 974.732 BIC: 1002.350


We can see that all criteria used AIC, AICc, and BIC for model 2 are smaller then those criteria used for model 1a and 1b. Given that, we can conclude that the Lasso Regression with tunned alpha presented a better fit.

The difference in BIC and AIC formula is the factor that multiplies the number of parameters used in the model. 

- bic = n * log(mse) + num_params * log(n)
- aic = n * log(mse) + 2 * num_params

Log(100) equals to 2. That means that BIC will be higher than AIC for every model that was trained using a dataset with more than 100 rows.

The AICc in all models was slightly higher than the AIC.

## RCT

**What are Randomized Control Trials: A/B Tests? What is its significance? Elaborate. Explain methods used in analyzing the RCT data?**

At its fundamental, the A/B test is a methodology used to measure the effects of a given change among two groups: a group with no difference (control) and a group with the difference (treatment).

In the context of the A/B test, statistical significance is the threshold used to decide whether an effect was caused by the treatment or only by chance. For example, when running a test using a 5% significance level, we assume that results with a p-value lesser than 0.05 are rare, occurring most likely by the effects of the treatment. On the other hand, results with a p-value greater than 0.05 are not that rare, so there is no significant treatment effect.

We have three main methods used in analyzing the RCT data:

1. Longitudinal analysis of covariance: we analyze the outcome variable at different times, adjusting it for the baseline value of the outcome.

2. Repeated measures: instead of adjusting for the baseline, in the repeated measures, we use the outcome of all three values (baseline, t1, and t2) as outcomes of the analysis.

3. Analysis of changes: in this method, we analyze not the values themselves but the changes between first measurement and baseline and second measurements and baseline.